In [1]:
from ultralytics import YOLO
import os
import random
import cv2
import numpy as np
import shutil
import supervision as sv
import cv2
import matplotlib.pyplot as plt




/home/aakesh/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
# Choose dataset 1 or 2:
dataset = 1  # 1 or 2

if dataset == 1:
    data_folder = "1_train-val_1min_aalesund_from_start/"
    yaml_file_combined = "yaml_files/data_1.yaml"
    yaml_file_ball = "yaml_files/ball_1.yaml"
    yaml_file_players = "yaml_files/player_1.yaml"

else:
    data_folder = "2_train-val_1min_after_goal/"
    yaml_file_combined = "yaml_files/data_2.yaml"
    yaml_file_ball = "yaml_files/ball_2.yaml"
    yaml_file_players = "yaml_files/player_2.yaml"

test_combined = "yaml_files/test_files/test_combined.yaml"
test_ball = "yaml_files/test_files/test_ball.yaml"
test_players = "yaml_files/test_files/test_players.yaml"

img_width = 1920  
img_height = 1080  

In [5]:
# Generate shuffled txt and img files for training and validation, 80% training and 20% validation

from scripts.shuffler import generate_shuffled_txt_img_files_for_train_val
from scripts.ball_dataset import generate_ball_dataset
from scripts.player_dataset import generate_player_dataset

object_path = f"data_yolov8/object_datasets/{data_folder}/"  
ball_path = f"data_yolov8/ball_datasets/{data_folder}/"  
player_path = f"data_yolov8/player_datasets/{data_folder}/"
text_file_path = f"/datasets/tdt4265/other/rbk/{data_folder}/gt/gt.txt"  
all_images_path = f"/datasets/tdt4265/other/rbk/{data_folder}/img1"



generate_shuffled_txt_img_files_for_train_val(text_file_path, img_width, img_height, object_path, all_images_path)
generate_ball_dataset(text_file_path, img_width, img_height, ball_path, all_images_path)
generate_player_dataset(text_file_path, img_width, img_height, player_path, all_images_path)


In [6]:
# Generate test txt and img files

from scripts.test_dataset import generate_txt_img_files_for_test

object_path = f"data_yolov8/object_datasets/3_test_1min_hamkam_from_start/"  
ball_path = f"data_yolov8/ball_datasets/3_test_1min_hamkam_from_start/"  
player_path = f"data_yolov8/player_datasets/3_test_1min_hamkam_from_start/" 
text_file_path = "/datasets/tdt4265/other/rbk/3_test_1min_hamkam_from_start/gt/gt.txt"  
all_images_path = "/datasets/tdt4265/other/rbk/3_test_1min_hamkam_from_start/img1/"

generate_txt_img_files_for_test(text_file_path, img_width, img_height, object_path, all_images_path)
generate_txt_img_files_for_test(text_file_path, img_width, img_height, ball_path, all_images_path, type='ball')
generate_txt_img_files_for_test(text_file_path, img_width, img_height, player_path, all_images_path, type='player')

In [3]:
# Last inn sist trente modell

def find_latest_model_with_best(base_path, type):
    detect_path = os.path.join(base_path, 'runs/detect', type)
    training_sessions = [os.path.join(detect_path, d) for d in os.listdir(detect_path) if os.path.isdir(os.path.join(detect_path, d))]
    
    # Liste for å holde stiene til alle 'best.pt'-filer
    best_model_paths = []

    for session in training_sessions:
        best_model_path = os.path.join(session, 'weights', 'best.pt')
        if os.path.exists(best_model_path):
            best_model_paths.append((best_model_path, os.path.getmtime(best_model_path)))

    if not best_model_paths:
        print("Ingen 'best.pt' fil funnet i noen av treningsøktene.")
        return None
    
    # Sorter basert på modifiseringstid og returner den nyeste
    best_model_paths.sort(key=lambda x: x[1], reverse=True)
    return best_model_paths[0][0]


base_path = '/work/aakesh/TDT4265_Project' 

#model_combined = YOLO(find_latest_model_with_best(base_path, 'combined'))
#model_ball = YOLO(find_latest_model_with_best(base_path, 'ball'))
#model_players = YOLO(find_latest_model_with_best(base_path, 'player'))
model = YOLO('yolov8n.pt')


In [5]:
# Object Detection

# Tren modellen på datasettet
#results = model_combined.train(data=yaml_file_combined, epochs=2, batch=8, imgsz=(1920, 1080), project='/work/mbergst/TDT4265_Project/runs/detect/combined')
results = model.train(data=yaml_file_ball, cfg='/work/aakesh/TDT4265_Project/runs/detect/tune2/best_hyperparameters.yaml', epochs=100, batch=14, imgsz=1920, project='/work/aakesh/TDT4265_Project/runs/detect/ball')
#results = model_players.train(data=yaml_file_players, epochs=100, batch=8, imgsz=(1920, 1080), project='/work/mbergst/TDT4265_Project/runs/detect/player')

#model.tune(data=yaml_file_ball, epochs=30, iterations=100, optimizer='AdamW', imgsz=1920)



Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=yaml_files/ball_1.yaml, epochs=100, time=None, patience=100, batch=14, imgsz=1920, save=True, save_period=-1, cache=False, device=None, workers=8, project=/work/aakesh/TDT4265_Project/runs/detect/ball, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, sa

train: Scanning /work/aakesh/TDT4265_Project/data_yolov8/ball_datasets/1_train-val_1min_aalesund_from_start/labels/train.cache... 1024 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1024/1024 [00:00<?, ?it/s]
val: Scanning /work/aakesh/TDT4265_Project/data_yolov8/ball_datasets/1_train-val_1min_aalesund_from_start/labels/val.cache... 257 images, 0 backgrounds, 0 corrupt: 100%|██████████| 257/257 [00:00<?, ?it/s]


Plotting labels to /work/aakesh/TDT4265_Project/runs/detect/ball/train5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.00718' and 'momentum=0.93197' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0006015625), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1920 train, 1920 val
Using 8 dataloader workers
Logging results to /work/aakesh/TDT4265_Project/runs/detect/ball/train5
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/74 [00:00<?, ?it/s]UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
      1/100      17.7G      1.303      47.92      1.089          5       1920: 100%|██████████| 74/74 [00:18<00:00,  3.97it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:01<00:00,  6.01it/s]

                   all        257        257      0.992      0.465      0.723      0.386


...

    100/100      17.5G     0.6317     0.4389      0.991          2       1920: 100%|██████████| 74/74 [00:17<00:00,  4.34it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:00<00:00, 11.05it/s]

                   all        257        257      0.986      0.946       0.99      0.694



100 epochs completed in 0.513 hours.
Optimizer stripped from /work/aakesh/TDT4265_Project/runs/detect/ball/train5/weights/last.pt, 6.5MB
Optimizer stripped from /work/aakesh/TDT4265_Project/runs/detect/ball/train5/weights/best.pt, 6.5MB

Validating /work/aakesh/TDT4265_Project/runs/detect/ball/train5/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:01<00:00,  7.43it/s]


                   all        257        257      0.976      0.969      0.992      0.712
Speed: 0.3ms preprocess, 2.1ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /work/aakesh/TDT4265_Project/runs/detect/ball/train5


In [ ]:
#model_combined = YOLO("/work/mbergst/TDT4265_Project/runs/detect/combined/train7/weights/best.pt")
model_combined.predict('/work/mbergst/TDT4265_Project/data_yolov8/object_datasets/1_train-val_1min_aalesund_from_start/images/all_frames/000763.jpg', 
                       save=True, imgsz=(1920, 1080), conf=0.5, project='/work/mbergst/TDT4265_Project/runs/detect/combined')

In [22]:
# Evaluér modellens prestasjon på valideringssettet
# Gjør det litt ulikt train sitt output pga ingen parametere definert
#val_results = model_combined.val()


In [9]:
# Test modellen på testsettet
#test_combined = model_combined.val(data=test_combined, batch=8, imgsz=(1920, 1080), project = '/work/mbergst/TDT4265_Project/runs/detect/combined/test')
test_ball = model.val(data=test_ball, batch=14, imgsz=1920, project = '/work/aakesh/TDT4265_Project/runs/detect/ball/test')
#test_players = model_players.val(data=test_players, batch=8, imgsz=(1920, 1080))


WARNING ⚠️ updating to 'imgsz=1920'. 'train' and 'val' imgsz must be an integer, while 'predict' and 'export' imgsz may be a [h, w] list or an integer, i.e. 'yolo export imgsz=640,480' or 'yolo export imgsz=640'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)


val: Scanning /work/aakesh/TDT4265_Project/data_yolov8/ball_datasets/3_test_1min_hamkam_from_start/labels/test... 1412 images, 390 backgrounds, 0 corrupt: 100%|██████████| 1802/1802 [00:00<00:00, 6994.83it/s]

val: New cache created: /work/aakesh/TDT4265_Project/data_yolov8/ball_datasets/3_test_1min_hamkam_from_start/labels/test.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/226 [00:00<?, ?it/s]UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 226/226 [00:08<00:00, 26.26it/s]


                   all       1802       1412      0.859      0.643      0.723      0.466
Speed: 0.4ms preprocess, 3.1ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /work/aakesh/TDT4265_Project/runs/detect/ball/test/train53


In [6]:
# Object tracking
all_frames = f'/datasets/tdt4265/other/rbk/{data_folder}/img1'

frame_paths = sorted([os.path.join(all_frames, f) for f in os.listdir(all_frames) if f.endswith('.jpg')])


In [7]:
# Iterere over hver frame og bruk track metoden
for frame_path in frame_paths:
    # model.track kan være en generator, så vi bruker en løkke for å hente resultatene
    for result in model_combined.track(frame_path, persist=True, stream=True, line_width=1):
        annotated_frame = result.plot(font_size=1, line_width=1)
        cv2.imshow('frame', annotated_frame)
        # Vent på et tastetrykk i 1 millisekund og sjekk om det er 'ESC'
        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # 27 er ASCII-verdien for 'ESC'
            break

    if key == 27:  # Kontroller igjen om 'ESC' ble trykket for å bryte ut av den ytre løkken
        break

cv2.destroyAllWindows()  # Lukker alle OpenCV-vinduene


image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000001.jpg: 1088x1920 1 ball, 22 players, 45.1ms
Speed: 4.4ms preprocess, 45.1ms inference, 419.2ms postprocess per image at shape (1, 3, 1088, 1920)

image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000002.jpg: 1088x1920 1 ball, 22 players, 2.5ms
Speed: 4.5ms preprocess, 2.5ms inference, 0.5ms postprocess per image at shape (1, 3, 1088, 1920)

image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000003.jpg: 1088x1920 1 ball, 22 players, 2.8ms
Speed: 2.7ms preprocess, 2.8ms inference, 0.6ms postprocess per image at shape (1, 3, 1088, 1920)

image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000004.jpg: 1088x1920 1 ball, 22 players, 2.5ms
Speed: 3.6ms preprocess, 2.5ms inference, 0.7ms postprocess per image at shape (1, 3, 1088, 1920)

image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000005.

In [9]:
# Blender frames fra 2 modeller

for frame_path in frame_paths:
    # Hent resultatene fra begge modellene for hver frame
    for result1, result2 in zip(model_players.track(frame_path, persist=True, stream=True, line_width=1),
                                model_ball.track(frame_path, persist=True, stream=True, line_width=1)):
        # Tegn annotasjoner fra begge modellene på samme bilde
        annotated_frame1 = result1.plot(font_size=1, line_width=1)
        annotated_frame2 = result2.plot(font_size=1, line_width=1)

        combined_frame = cv2.addWeighted(annotated_frame1, 0.5, annotated_frame2, 0.5, 0)  

        # Vis det annoterte bildet med resultater fra begge modeller
        cv2.imshow('Combined frame', combined_frame)
        
        # Vent på et tastetrykk i 1 millisekund og sjekk om det er 'ESC'
        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # 27 er ASCII-verdien for 'ESC'
            break

    if key == 27:  # Kontroller igjen om 'ESC' ble trykket for å bryte ut av den ytre løkken
        break

cv2.destroyAllWindows()  # Lukker alle OpenCV-vinduene



image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000001.jpg: 1088x1920 22 players, 2.5ms

image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000001.jpg: 1088x1920 1 ball, 2.5ms
Speed: 3.0ms preprocess, 2.5ms inference, 0.5ms postprocess per image at shape (1, 3, 1088, 1920)

image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000002.jpg: 1088x1920 22 players, 2.4ms

image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000002.jpg: 1088x1920 1 ball, 2.3ms
Speed: 4.4ms preprocess, 2.4ms inference, 0.5ms postprocess per image at shape (1, 3, 1088, 1920)

image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000003.jpg: 1088x1920 22 players, 2.4ms

image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000003.jpg: 1088x1920 1 ball, 2.6ms
Speed: 3.1ms preprocess, 2.4ms inference, 0.6ms postprocess per image at shape (1, 3, 1088

In [26]:
# Object tracking with supervision

boxann = sv.BoxAnnotator(
    thickness=1,
    text_thickness=1,
    text_scale=0.5,
    
)

for frame_path in frame_paths:
    # model.track kan være en generator, så vi bruker en løkke for å hente resultatene
    for result in model.track(frame_path, persist=True, stream=True, line_thickness=1):
        frame = result.orig_img
        
        detections = sv.Detections.from_ultralytics(result)
        

        frame = boxann.annotate(scene=frame, detections=detections)
        cv2.imshow('yolov8', frame)
        cv2.waitKey(1)  # Venter en kort tid så bildet kan oppdateres

cv2.destroyAllWindows()  # Lukker alle OpenCV-vinduene

WARNING ⚠️ 'line_thickness' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'line_width' instead.

image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/1_train-val_1min_aalesund_from_start/images/all_frames/000001.jpg: 1088x1920 22 players, 4.5ms


SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


error: OpenCV(4.9.0) /io/opencv/modules/highgui/src/window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [ ]:
# Not working Object tracking

for frame in frame_paths:
    results = model.track(frame, persist=True)
    boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
    ids = results[0].boxes.id.cpu().numpy().astype(int)
    for box, id in zip(boxes, ids):
        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
        cv2.putText(
            frame,
            f"Id {id}",
            (box[0], box[1]),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2,
        )
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break


image 1/1 /work/mbergst/project/data_yolov8/1_train-val_1min_aalesund_from_start/images/all_frames/000001.jpg: 1088x1920 2 players, 3.2ms
Speed: 21.6ms preprocess, 3.2ms inference, 0.6ms postprocess per image at shape (1, 3, 1088, 1920)


error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'
